In [4]:
from code_tokenizer import *

code = """
a = 1
if(a == 1)
{
    a = 2
}
"""

print(color_by_char_type(code))

for item in parse(code):
    print(item)


console.log(a)

['access_group', [['word', 'console'], ['word', 'log', ['attached_group', ['word', 'a']]]]]
